# Basic probability

Python activities to complement [*Measurements and their Uncertainties*](http://www.oupcanada.com/catalog/9780199566334.html) (*MU*), Chapter 3, "Uncertainties as probabilities."

* [Preliminaries](#Preliminaries)
* [Probability calculations](#Probability-calculations)
    * [Probability calculations with the uniform distribution](#Probability-calculations-with-the-uniform-distribution)
        * [Programming notes 1](#Programming-notes-1)
        * [Programming notes 2](#Programming-notes-2)
        * [Programming notes 3](#Programming-notes-3)
        * [Programming notes 4](#Programming-notes-4)
    * [Exercise 1](#Exercise-1)
    * [Exercise 2](#Exercise-2)
    * [Probability calculations with the normal distribution](#Probability-calculations-with-the-normal-distribution)
        * [Programming notes 5](#Programming-notes-5)
    * [Exercise 3](#Exercise-3)
    * [Outliers, or: Chauvenet is dead. Let his criterion die with him.](#Outliers,-or&#58;-Chauvenet-is-dead.-Let-his-criterion-die-with-him.)
    * [Exercise 4](#Exercise-4)
* [The Poisson distribution](#The-Poisson-distribution)
    * [Distribution plot](#Distribution-plot)
        * [Programming notes 6](#Programming-notes-6)
    * [Exercise 5](#Exercise-5)
        * [Programming notes 7](#Programming-notes-7)
        * [Programming notes 8](#Programming-notes-8)
* [The central limit theorem](#The-central-limit-theorem)
    * [Exercise 6](#Exercise-6)
* [Summary](#Summary)
* [Further reading](#Further-reading)

## Preliminaries
Before proceeding with this notebook you should review the topics from the [previous notebook](2.0-Basic-statistics.ipynb) and read *MU* Ch. 3, "Uncertainties as probabilities," with the following [goals](https://wiki.its.sfu.ca/departments/phys-students/index.php/Reading_goals_for_Hughes_and_Hase#Uncertainties_as_probabilities) in mind.

1. Be able to explain what a probability distribution function $P_\text{DF}(x)$ represents and why Eqs. (3.1) - (3.6) follow from its definition.
2. Be able to recall and use Eqs. (3.1) - (3.3) to perform simple probability calculations for an arbitrary $P_\text{DF}(x)$, including:
    1. Check that $P_\text{DF}(x)$ is properly normalized, and identify the correct normalization factor if it is not;
    2. Evaluate the expectation value of a function $f(x)$; and
    3. Evaluate the expectation value of the mean and the variance.
3. Be able to recall the definitions (3.7) and (3.8) of the Gaussian probability distribution function and the error function, respectively, and know how to use the error function in simple probability calculations like the one given in Sec. 3.2.2.
4. Be aware of the rules described in Sec. 3.3.2 for rejecting outliers, be able to follow a well-defined procedure for doing so, and be able to suggest alternatives to throwing away data points.
5. Be able to describe the basic properties of a Poisson distribution $P(N;\bar{N})$, including:
    1. its functional form;
    2. the kind of experimental data that will be described by it;
    3. the expectation values of its mean and variance; and
    4. the Gaussian probability distribution that approximates it for $N\rightarrow\infty$.
6. Be able to sketch a Poisson distribution for a given mean and standard deviation, and be able to estimate the mean and standard deviation from the plot of a Poisson distribution.
7. Be able to state the central limit theorem and recognize how it is used to justify the assumption of Gaussian errors in many experiments.

The following code cell includes the usual initialization commands, updated to load the normal distribution object `norm`.

In [ ]:
import numpy as np
from numpy import random
from scipy.stats import norm
import matplotlib.pyplot as plt

%matplotlib inline

## Probability calculations
*MU* Sec. 3.1 lists three integral expressions that describe important properties of a continuous PDF.

Eq. (3.1) is a *normalization condition* that ensures that the probability summed over all possible outcomes is one:

<a id="MU(3.1)"></a>$$ \int_{-\infty}^{\infty}\text{d}x\,P_\text{DF}(x) = 1.$$

Eq. (3.2) allows us to predict the probabilities of specific outcomes:

<a id="MU(3.2)"></a>$$P(x_1 \le x \le x_2) = \int_{x_1}^{x_2}\text{d}x\,P_\text{DF}(x).$$

Eq. (3.3) expresses the *expectation* of $x^n$, given by the weighted average

<a id="MU(3.3)"></a>$$ \left\langle x^n\right\rangle = \int_{-\infty}^{\infty}\text{d}x\,x^n P_\text{DF}(x), $$

where we use the $\langle x^n\rangle$ notation discussed in the [previous notebook](2.0-Basic-statistics.ipynb#Determining-the-mean-and-variance-from-the-distribution) instead of the overline notation $\overline{x^n}$ used in *MU*. This quantity is also called the $n$th moment of $x$.

We can also generalize Eq. (3.3) to describe the expectation of an arbitrary *function*, $f(x)$,

<a id="<f(x)>"></a>$$ \left\langle f(x)\right\rangle = \int_{-\infty}^{\infty}\text{d}x\,f(x) P_\text{DF}(x). $$

Finally, *MU* Eq. (3.6) gives a useful identity, proven in *MU* Eq. (3.5),

<a id="MU(3.6)"></a>$$ \sigma^2 = \langle(x-\mu)^2\rangle = \langle x^2\rangle - \langle x\rangle^2. $$

Usually we will evaluate these expressions numerically, not analytically, and there are several SciPy routines to help with this.

### Probability calculations with the uniform distribution
The uniform distribution is appropriate for measurements with discrete precision (see *MU*, Sec. 1.3.2), and you evaluated the integral in *MU* Eq. (3.2) by hand for the standard uniform distribution $\mathcal{U}(x;0,1)$ in [Exercise  3](2.0-Basic-statistics.ipynb#Exercise-3) of the last notebook. The following code cell shows how to do this numerically for $P(0.0 \le x \le 0.5)$.

#### Programming notes 1
The first line imports the [`uniform`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) distribution object from [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html), and the second line imports [`quad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html), which is short for [quadrature](https://en.wikipedia.org/wiki/Quadrature_(mathematics)), from [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html). The `quad` routine returns two numbers: the value of the integral, assigned here to `P`, and an estimate of its absolute error, assigned to `P_err`.

In [ ]:
from scipy.stats import uniform
from scipy.integrate import quad

P, P_err = quad(uniform.pdf, 0.0, 0.5)
print("P = ", P)
print("P_err = ", P_err)

In the next code cell we compute the mean by evaluating *MU* Eq. (3.4),

<a id="MU(3.4)"></a>$$ \left\langle x\right\rangle = \int_{-\infty}^{\infty}\text{d}x\,x \mathcal{U}(x;0,1). $$

We then compute the second moment by evaluating [*MU* Eq. (3.3)](#MU(3.3)) with $n=2$, and the variance by evaluating [*MU* Eq. (3.6)](#MU(3.6)).

#### Programming notes 2
The `quad` routine requires a [function](https://docs.python.org/3/tutorial/controlflow.html#defining-functions) of a single argument to integrate, which we define in the first two statements below. It has the structure

    def <name>(<input args>):
        <intermediate statements>
        return <output args>

Here, the function *name* is `mean_int`, and it has a single input argument `x`. This is a simple function, so we do not need to include any intermediate statements; we just `return` the single output argument given by `x*uniform.pdf(x)`.

In [ ]:
# Define integrand for the mean
def mean_int(x):
    return  x*uniform.pdf(x)

# Integrate over domain and print result
xbar, xbar_err  =  quad(mean_int, 0.0, 1.0)
print("Mean: ", xbar)

# Repeat for the second moment
def m2_int(x):
    return  x**2*uniform.pdf(x)

m2, m2_err  =  quad(m2_int, 0.0, 1.0)
print("Second moment: ", m2)

# Repeat for the variance
def var_int(x):
    return  (x - 0.5)**2*uniform.pdf(x)

var, var_err  =  quad(var_int, 0.0, 1.0)
print("Variance: ", var)

#### Programming notes 3
The `scipy.stats` package also includes [methods](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html#common-methods) for simple statistics like these, as demonstrated in the next code cell. The `mean`, `std`, and `var` methods are listed at the end of the help file for the [`uniform`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) distribution object, and you can use `?` to get additional help for each of them. For example, type `uniform.mean?` in a new code cell to see the help for `mean`.

In [ ]:
# Get mean, std, var, and 2nd moment
u_mean = uniform.mean()
u_std = uniform.std()
u_var = uniform.var()
u_m2  = uniform.moment(2)

print("Statistics for the standard uniform distribution")
print("================================================")
print("Mean: ", u_mean)
print("Standard deviation: ", u_std)
print("Variance:  ", u_var)
print("Second moment: ", u_m2)

#### Programming notes 4
We can also use the `loc` and `scale` keywords to get information about more general uniform distributions. These keywords have different meanings for different distributions, so check the documentation. With [`uniform`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html), the distribution interval is given by `[loc, loc + scale]`.

In [ ]:
# Define U(0,2)
# Lower bound is `loc`, width is `scale`
loc = -1.0
scale = 2.0
def ugen(x):
    return uniform.pdf(x, loc, scale)

# Compute P for x in [0.0, 0.5]
Pab, Pab_err = quad(ugen, 0.0, 0.5)
print("P in [0.0, 0.5]: ", Pab)
print()

# Get mean, std, var, 2nd moment
uab_mean = uniform.mean(loc=-1.0, scale=2.0)
uab_std = uniform.std(loc=-1.0, scale=2.0)
uab_var = uniform.var(loc=-1.0, scale=2.0)
uab_m2 = uniform.moment(2, loc=-1.0, scale=2.0)

print("Statistics for the general uniform distribution")
print("with loc = -1.0 and scale = 2.0")
print("================================================")
print("Mean: ", uab_mean)
print("Standard deviation: ", uab_std)
print("Variance:  ", uab_var)
print("Second moment: ", uab_m2)

### Exercise 1
A digital scale tells me that my morning coffee beans have a mass of 25.0 grams. What is the standard deviation of the parent distribution for this measurement? Compare this to the recommendation in *MU*, Sec. 1.3.2, that we report digital uncertainties as the smallest digital increment. Either this convention or the *mean* ± *standard deviation* convention are acceptable ways to report the uncertainty, but it is important to recognize that they are different and to communicate clearly which one you are using.

In [ ]:
# Code cell for Exercise 1
# Use this cell for your response, adding cells if necessary.

### Exercise 2
Show analytically that for the general uniform distribution $P_\text{DF}(x) = \mathcal{U}(x;a,b)$,

$$ \left\langle(x-\mu)^2\right\rangle = \frac{(b-a)^2}{12}. $$

**Markdown cell for Exercise 2**

Select this cell and enter your response here.

### Probability calculations with the normal distribution
By far the most common probability calculations in data analysis involve the normal distribution. Other than the [normalization integral](2.1-Evaluating-the-Gaussian-integral.ipynb), we must rely on numerical methods to evaluate definite integrals over Gaussian integrands. Below we demonstrate techniques for reproducing one of the example calculations in *MU* Sec. 3.2.2.

**Important: The definition of the *error function* used in *MU* is not standard.** The function defined in *MU* Eq. (3.8) is more commonly called the [*cumulative distribution function*](https://en.wikipedia.org/wiki/Normal_distribution#Cumulative_distribution_function) (CDF) for the normal distribution, and is represented by the Greek letter $\Phi$. We will adopt this more standard notation here (including the choice of $\sigma^2$ instead of $\sigma$ to parameterize it) and rewrite Eq. (3.8) as

<a id="MU(3.8)"></a>$$ \Phi(x_1; \mu, \sigma^2) = \int_{-\infty}^{x_1}\text{d}x\,\mathcal{N}(x; \mu, \sigma^2). $$

The more usual definition of the [*error function*](https://en.wikipedia.org/wiki/Error_function) is

<a id="erf"></a>$$ \text{erf}(x) = \frac{2}{\sqrt{\pi}}\int_{0}^{x}\text{d}t\,e^{-t^2} = \frac{1}{\sqrt{\pi}}\int_{-x}^{x}\text{d}t\,e^{-t^2},$$

which gives the area under $\mathcal{N}(\mu=0,\sigma^2 =  1/2)$ between $-x$ and $x$. (The dummy variable $t$ is also commonly used in the standard definition, so we have preserved that here. *MU* uses $x$ as the dummy variable and $x_1$ as the argument to the cumulative distribution function, which we also preserve.)

This, more standard definition of the error function is related to $\Phi$ through

<a id="erf2cdf"></a>\begin{align}
\Phi(x_1; \mu, \sigma^2) &= \frac{1}{\sqrt{2\pi\sigma^2}}\int_{-\infty}^{x_1}\text{d}x\,\exp\left[-\frac{(x-\mu)^2}{2\sigma^2}\right]\\
&= \frac{1}{\sqrt{\pi}}\int_{-\infty}^{t_1}\text{d}t\,e^{-t^2},\quad \text{with}\ t_1 = \frac{x_1-\mu}{\sqrt{2}\sigma}\\
&= \frac{1}{2}\left[1 + \text{erf}\left(\frac{x_1 - \mu}{\sqrt{2}\sigma}\right)\right].
\end{align}

Now we are ready to reproduce the first example calculation in *MU* Sec. 3.2.2, where we have a box of resistors that are normally distributed with $\mu = 100~\Omega$ and $\sigma = 2~\Omega$, and we want to determine the probability of picking a resistor with a resistance less than 95&nbsp;&Omega;.

#### Programming notes 5
We import the error function [`erf`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.html) from [`scipy.special`](https://docs.scipy.org/doc/scipy/reference/special.html), the special functions package of SciPy. We use the `cdf` method of the normal distribution object [`norm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) from `scipy.stats`. The result using `erf` and `norm.cdf` are within the numerical precision of each other.

In [ ]:
# Import error function
from scipy.special import erf

# Assign variables
mu_R = 100
sigma_R = 2
R1 = 95

# Compute scaled variable t1_a for the error function
t1 = (R1 - mu_R)/(np.sqrt(2)*sigma_R)

# Evaluate CDF using the error function
P_erf = (1 + erf(t1))/2
print("Probability (erf): ", P_erf)

# Evaluate CDF using the cdf method of norm
P_cdf = norm.cdf(R1, loc=mu_R, scale=sigma_R)
print("Probability (cdf): ", P_cdf)

### Exercise 3
Reproduce the second example in *MU* Sec. 3.2.2; that is, for a box of resistors that are normally distributed with $\mu = 100~\Omega$ and $\sigma = 2~\Omega$, determine the probability of picking a resistor with a resistance between 99&nbsp;&Omega; and 101&nbsp;&Omega;.

In [ ]:
# Code cell for Exercise 3
# Use this cell for your response, adding cells if necessary.

### Outliers, or: Chauvenet is dead. Let his criterion die with him.
You now have the tools to confirm the assertion in *MU* Sec. 3.3.2, that "the fractional area under a Gaussian curve beyond $3\sigma$ or $5\sigma$ is only 0.3% or $6\times 10^{-5}\%$, respectively." These are small probabilities, and over time they have been used in the particle physics community to establish standards for "evidence" and "discovery" of a new particle, such as the Higgs boson. (Technically, with the Higgs discovery the associated probability was actually $3\times 10^{-5}\%$, the probability of being $5\sigma$ *above* the background, not counting the probability of being $5\sigma$ below it.)

When we observe something that we expect to happen by chance less than once in a million, we tend to believe that it didn't happen by chance. But our expectation that measurements should follow a normal distribution is often wrong. One [study](#Bailey2017) of the published literature found that $5\sigma$ events occurred up to *100,000 times more frequently than expected* for normally distributed measurements. What the normal distribution suggests should be a one-in-a-million event might actually be one-in-ten! This large discrepancy may be traced back to differences between the "tails" of different distributions, which refers to the portions of the distribution that lie far from the mean. The exponential form of the normal distribution causes it to approach zero much more rapidly than real measurements usually do.

So despite what the normal distribution tells you, your measurements will turn out differently from what you expect more frequently than you expect. Sometimes this can be the seed of a discovery, but more often than not you will have neglected something that turns out to be important. The best response to this situation is to investigate further, take more data, and figure out what you missed. There are many places to look—here are some common ones:
* Sources of systematic error in the experiment;
* Unrecognized sources of measurement variability;
* Limitations in the mathematical model used to describe the experiment;
* Mistakes.

Chauvenet's criterion, on the other hand, discussed in *MU* Sec. 3.3.2, provides an elaborate method for you to *just pretend your little outlier never happened.* I'm sure Chauvenet meant well, but this really is a misguided approach to the problem—and Hughes and Hase appear to have included it in their book for no reason other than a grudging sense of obligation. Rather than focusing attention on what you might *learn* from an anomalous measurement, Chauvenet's approach focuses on *how to get rid of it.* Moreover, it is easy to [show](#Barnett1994) that as the number of measurements increases, the probability that you Chauvenet's criterion will recommend rejecting a perfectly reasonable result approaches 40%! Even if it's 6 am, your lab report is due in two hours, and you have no hope of figuring out what might have caused the outlier, you can still report your results with and without it and draw conclusions from the comparison. Chauvenet's criterion just provides a pretence of objectivity that obscures more than it clarifies. Don't use it.

### Exercise 4
Confirm that the probability of obtaining a result $3\sigma$ away or more from the mean is $P_{3\sigma} = 2.7\times 10^{-3}$, and $5\sigma$ away is $P_{5\sigma} = 5.7\times 10^{-7}$.

In [ ]:
# Code cell for Exercise 4
# Use this cell for your response, adding cells if necessary.

## The Poisson distribution
*MU* Sec. 3.4 discusses the Poisson distribution, a discrete distribution that appears commonly in counting experiments. We reproduce some of the results of that section below.

### Distribution plot
First, we reproduce *MU* Fig. 3.5(a)., the probability distribution for mean $\bar{N}=1.5$. Note that the vertical axis is dimensionless now, because the distribution assigns discrete probabilities to each discrete outcome, unlike the probability density for a continuous distribution.

#### Programming notes 6
We import the Poisson distribution object, [`poisson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html), then use its `pmf` method to compute its probability distribution, also called the *probability mass function.* We then use the [`bar`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.bar.html?highlight=bar) function in pyplot to produce the bar plot and format it.

In [ ]:
# Import Poisson distribution object
from scipy.stats import poisson

# Define variables
n = np.arange(8)
n_bar = 1.5

# Generate distribution
p = poisson.pmf(n, n_bar)

plt.bar(n, p, ec='k')
plt.xlim([-0.5, 8.5])
plt.xlabel('Number of counts, n')
plt.ylabel('Probability');

### Exercise 5
Reproduce below *MU* Fig. 3.5(b), which shows the Poisson probability distribution for mean $\bar{N}=15$.

In [ ]:
# Code cell for Exercise 5
# Use this cell for your response, adding cells if necessary.

Next we confirm, for $\lambda = 10$ (using $\mu$ instead of $\bar{N}$), the expectations for the mean, *MU* Eq. (3.12),

$$ \langle N\rangle = \sum_{N=0}^{\infty} P(N;\lambda) N = \lambda, $$

and second moment, *MU* Eq. (3.13),

$$ \langle N^2\rangle = \sum_{N=0}^{\infty} P(N;\lambda) N^2 = \lambda(\lambda + 1), $$

and the variance,

$$ \langle N^2\rangle - \langle N\rangle^2 = \lambda.$$

#### Programming notes 7
We compute these quantities in two different ways. First, we use the `mean`, `moment`, and  `var` methods of the Poisson distribution object [`poisson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html). Then we use its `pmf` method to compute its probability distribution up to an upper limit `n_max = 100` and use this to evaluate the sums directly. Note that `lambda` is a reserved keyword in Python so we use `lam` to represent $\lambda$.

In [ ]:
# Set the distribution mean
lam = 10

# Show expectations with poisson methods
print("Mean: ", poisson.mean(lam))
print("Second moment: ", poisson.moment(2, lam))
print("Variance: ", poisson.var(lam))

# Compute expectations with sums up to n_max = 100
n_max = 100
n = np.arange(n_max)
n_mean = np.sum(poisson.pmf(n, lam)*n)
n_m2 = np.sum(poisson.pmf(n, lam)*n**2)

print("Mean (sum): ", n_mean)
print("Second moment (sum): ", n_m2)
print("Variance (sum): ", n_m2 - n_mean**2)

Now we evaluate the result of the worked example in *MU* Sec. 3.4.1. Initially it says that the trigger occurs when the count rate *exceeds* 13 counts per minute, but the calculation proceeds as if the trigger occurs with a count rate of *13 or more.* We compute the latter to compare with the result in *MU*.

#### Programming notes 8
Again, we compute these quantities in two different ways. First, we use the `cdf` method of [`poisson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html), following *MU*. Next we use the *survival function* method, `sf`, which computes `1 - cdf`. These yield identical results in this case, but the `sf` method is more accurate when `1 - cdf` is small. To see this, try setting `k_max = 50` and run the cell again.

In [ ]:
lambda_max = 1980/(10*60)
k_max = 12
P_stop_cdf = poisson.cdf(k_max, lambda_max)
P_stop_sf = poisson.sf(k_max, lambda_max)
print(f"Probability of exceeding {k_max:d} counts per minute (CDF): {1-P_stop_cdf:.2e}")
print(f"Probability of exceeding {k_max:d} counts per minute (SF): {P_stop_sf:.2e}")

# The central limit theorem

### Exercise 6
Run the code cell below with `n_avg = 1`, then increment `n_avg` and run it again. Repeat this procedure until you get to `n_avg = 5`, and notice how the distribution changes. Then, increase `n_avg` in incements of 5 and notice how the distribution changes, paying attention to both the scales and the shape.

In [ ]:
# Code cell for Exercise 6
# Generate a (n_avg, n_trial) array of U(0,1) random numbers
n_avg = 1
n_trial = 1000
random.seed(0)
x = np.random.rand(n_avg, n_trial)

# Calulate mean value along each column (axis=0)
x_bar = np.mean(x, axis=0)

plt.hist(x_bar, 20, ec='black')
plt.xlabel('Mean')
plt.ylabel('Occurrence');

## Summary
Here is a list of what you should be able to do after completing this notebook.
* Use distribution objects from [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html), specifically [`uniform`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html), [`norm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html), and [`poisson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html), including the `mean`, `std`, `var`, and `moment` methods to compute basic statistics, the `pdf` method to compute PDFs for continuous distributions, the `pmf` method to compute PMFs for discrete distributions, and the `cdf` and `sf` functions to compute the cumulative distribution function and the survival function, respectively
* Use [`quad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) to numerically evaluate integrals
* Define simple Python [functions](https://docs.python.org/3/tutorial/controlflow.html#defining-functions)
* Recognize the relationship between $\Phi(x;\mu,\sigma^2)$ and $\text{erf}(x)$ (using the standard definitions)
* Produce and format a bar plot with [`bar`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.bar.html?highlight=bar)
* Identify appropriate responses to a statistically significant disagreement between an experiment and expectations
* Recognize how averages of independent measurements tend toward a Gaussian distribution

## Further reading

<a id="Bailey2017"></a> David C. Bailey, ["Not Normal: the uncertainties of scientific measurements,"](http://dx.doi.org/10.1098/rsos.160600) R. Soc. open sci. **4**, 160600 (2017).

<a id="Bailey2018"></a> David Bailey, ["Why outliers are good for science,"](http://dx.doi.org/10.1111/j.1740-9713.2018.01105.x), Significance **15** (1), 14 (2018).

<a id="Barnett1994"></a> Vic Barnett and Toby Lewis, *Outliers in statistical data,* 3rd ed. (Wiley, West Sussex, 1994). More than you ever wanted to know.

##### About this notebook
Notebook by J. S. Dodge, 2019. Available from [SFU GitLab](https://gitlab.rcg.sfu.ca/jsdodge/data-analysis-python). The notebook text is licensed under the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License. See more at [Creative Commons](http://creativecommons.org/licenses/by-nc-nd/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).